# Class 정의

In [14]:
!python.exe -m pip install --upgrade pip
!pip install python-dotenv
!pip install google-generativeai
!pip install mysql-connector-python

  Using cached pip-25.0.1-py3-none-any.whl.metadata (3.7 kB)
Using cached pip-25.0.1-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 24.3.1
    Uninstalling pip-24.3.1:
      Successfully uninstalled pip-24.3.1
  Using cached mysql_connector_python-9.2.0-cp313-cp313-win_amd64.whl.metadata (6.2 kB)
Using cached mysql_connector_python-9.2.0-cp313-cp313-win_amd64.whl (16.1 MB)


In [ ]:
import os
import re
import json
from dotenv import load_dotenv
import google.generativeai as genai
import mysql.connector
from mysql.connector import errors

class NovelGenerator:
    def __init__(self, genre: str, title: str):
        # 환경변수 로드 및 API, DB 설정
        load_dotenv()
        self.GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
        genai.configure(api_key=self.GEMINI_API_KEY)

        self.genre = genre
        self.title = title
        self.worldview = ""
        self.synopsis = ""
        self.characters = ""
        self.first_chapter = ""
        self.next_chapter = ""
        
        # 여기서는 간단한 예시로 단일 소설에 대해 novel_pk를 1로 고정
        self.novel_pk = 1
        
        # DB 접속정보
        self.db_host = os.getenv("DB_HOST")
        self.db_port = int(os.getenv("DB_PORT", "3306"))
        self.db_user = os.getenv("DB_USER")
        self.db_password = os.getenv("DB_PASSWORD")
        self.db_name = os.getenv("DB_NAME")
        
        # 데이터베이스가 없는 경우 생성하도록 처리
        try:
            self.db_connection = mysql.connector.connect(
                host=self.db_host,
                port=self.db_port,
                user=self.db_user,
                password=self.db_password,
                database=self.db_name
            )
        except errors.ProgrammingError as e:
            if e.errno == 1049:  # Unknown database
                print(f"데이터베이스 {self.db_name}가 존재하지 않습니다. 데이터베이스를 생성합니다.")
                temp_conn = mysql.connector.connect(
                    host=self.db_host,
                    port=self.db_port,
                    user=self.db_user,
                    password=self.db_password
                )
                temp_cursor = temp_conn.cursor()
                temp_cursor.execute(f"CREATE DATABASE IF NOT EXISTS {self.db_name}")
                temp_conn.commit()
                temp_cursor.close()
                temp_conn.close()
                # 데이터베이스 생성 후 다시 연결
                self.db_connection = mysql.connector.connect(
                    host=self.db_host,
                    port=self.db_port,
                    user=self.db_user,
                    password=self.db_password,
                    database=self.db_name
                )
            else:
                raise e
        
        self.create_table_if_not_exists()

    def create_table_if_not_exists(self):
        """episode 테이블이 없으면 생성"""
        cursor = self.db_connection.cursor()
        query = """
        CREATE TABLE IF NOT EXISTS episode (
            ep_pk INT AUTO_INCREMENT PRIMARY KEY,
            ep_title VARCHAR(255) NOT NULL,
            novel_pk INT NOT NULL,
            created_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            updated_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
            ep_content TEXT NOT NULL
        )
        """
        cursor.execute(query)
        self.db_connection.commit()
        cursor.close()

    def insert_episode(self, ep_title: str, ep_content: str):
        """새 에피소드를 DB에 저장합니다."""
        cursor = self.db_connection.cursor()
        query = """
        INSERT INTO episode (ep_title, novel_pk, ep_content)
        VALUES (%s, %s, %s)
        """
        cursor.execute(query, (ep_title, self.novel_pk, ep_content))
        self.db_connection.commit()
        cursor.close()

    def get_previous_episodes(self) -> str:
        """해당 소설의 모든 에피소드(ep_content)를 created_date 기준 오름차순으로 불러와 하나의 문자열로 합칩니다."""
        cursor = self.db_connection.cursor()
        query = """
        SELECT ep_content FROM episode
        WHERE novel_pk = %s
        ORDER BY created_date ASC
        """
        cursor.execute(query, (self.novel_pk,))
        rows = cursor.fetchall()
        cursor.close()
        episodes = "\n\n---\n\n".join([row[0] for row in rows])
        return episodes

    def recommend_worldview(self) -> str:
        """소설 세계관 생성 함수"""
        instruction = """
        당신은 전문적으로 소설 세계관을 만드는 작가입니다.
        생성되는 텍스트는 순수한 일반 텍스트 형식이어야 하며, 어떠한 마크다운 문법(예: **, ## 등)도 사용하지 말아주세요.
        주어진 장르, 제목을 기반으로 독창적이고 생동감 있는 소설 세계관을 만들어주세요.
        소설 세계관을 구성할 때, 아래의 항목들을 참고하여 상세하게 작성하세요.

        1. 기본 설정
            - 시대: (예: 과거, 현대, 미래 등) - 시대적 배경과 분위기를 구체적으로 설명해주세요.
            - 핵심 테마: (예: 인간과 자연의 대립, 기술과 마법의 공존 등) - 세계관을 관통하는 핵심 주제를 명확히 제시하고, 다른 요소들과의 연관성을 설명해주세요.

        2. 지리적 배경
            - 세계 전체의 구조(대륙, 국가, 도시, 마을 등) - 지리적 구조를 시각적으로 표현하고, 각 지역의 특징을 설명해주세요.
            - 주요 지형 및 자연 환경(기후, 산맥, 강 등) - 자연 환경이 세계관에 미치는 영향을 설명하고, 상징적인 의미를 부여할 수 있습니다.
            - 상징적이거나 중요한 장소(예: 전설적인 성, 신비로운 숲 등) - 각 장소의 역사, 특징, 중요성 등을 구체적으로 설명해주세요.

        3. 역사와 문화
            - 세계의 건국 신화 및 주요 역사적 사건 - 역사적 사건이 세계관에 미친 영향과 현재 사회에 남은 유산을 설명해주세요.
            - 사회 구조(정치 체제, 경제 시스템, 계층 구조 등) - 사회 구조가 세계관 구성원들의 삶에 미치는 영향을 설명해주세요.
            - 문화와 종교(전통, 의식, 종교적 신념 등) - 문화와 종교가 세계관에 미치는 영향을 설명하고, 독특한 요소들을 강조해주세요.

        4. 기술 및 초자연적 요소
            - 세계 내 기술 수준(과거, 현대, 미래 기술) - 기술 수준이 세계관에 미치는 영향과 특징을 설명해주세요.
            - 만약 존재한다면 마법이나 초자연적 현상의 규칙과 한계 - 마법/초자연 현상이 세계관에 미치는 영향과 작동 방식을 구체적으로 설명해주세요.

        5. 인물 및 종족
            - 주요 인물이나 집단, 종족의 특징과 역할 - 각 인물/종족의 개성과 세계관 내 역할을 명확하게 설명해주세요.
            - 각 인물/종족이 속한 사회적, 문화적 배경 - 인물/종족의 배경이 그들의 행동 양식에 미치는 영향을 설명해주세요.

        6. 갈등 구조
            - 사회 내부 혹은 외부의 갈등 요소(정치적 분쟁, 종족 간 충돌 등) - 갈등의 원인, 진행 과정, 결과를 구체적으로 설명해주세요.
            - 이러한 갈등이 전체 세계관에 미치는 영향 - 갈등이 세계관의 다른 요소들에 미치는 영향을 설명해주세요.

        7. 세부 설정 및 고유 용어
            - 세계관 내에서만 사용되는 독특한 용어나 법칙 정리 - 용어의 의미와 사용 예시를 설명하여 세계관의 현실성을 높여주세요.
            - 설정의 일관성을 유지할 수 있도록 참고 자료 작성 - 필요한 경우, 추가적인 자료를 작성하여 세계관의 완성도를 높여주세요.
        """
        model = genai.GenerativeModel("models/gemini-2.0-flash", system_instruction=instruction)
        prompt = f"## 소설 장르: {self.genre}\n## 소설 제목: {self.title}\n\n**소설 세계관**\n"
        response = model.generate_content(prompt)
        self.worldview = response.text
        print("소설 세계관:\n", self.worldview)
        return self.worldview

    def recommend_synopsis(self) -> str:
        """소설 줄거리 생성 함수"""
        instruction = """
        당신은 전문적으로 소설 줄거리를 만드는 작가입니다.
        생성되는 텍스트는 순수한 일반 텍스트 형식이어야 하며, 어떠한 마크다운 문법(예: **, ## 등)도 사용하지 말아주세요.
        주어진 장르, 제목, 세계관을 기반으로 독창적이고 생동감 있는 소설 줄거리를 만들어주세요.
        소설 줄거리를 구성할 때, 아래의 항목들을 참고하세요.

        1. 기본 정보
            - 장르: [예: 판타지, SF 등]
            - 제목: [제목 입력]
            - 세계관: [세계관의 배경, 시대, 문화, 기술 등 간단히 설명]

        2. 도입부
            - 소설이 펼쳐지는 세계관 소개
            - 주요 인물 및 그들의 기본 목표
            - 초기 갈등이나 문제의 암시

        3. 전개부
            - 주된 갈등 및 도전 과제 전개
            - 인물 간의 관계와 서브 플롯 설명
            - 주인공이 맞서야 하는 문제와 성장 과정

        4. 클라이맥스
            - 갈등의 최고조와 결정적 순간
            - 주인공의 중요한 선택 및 대립의 절정

        5. 결말
            - 갈등 해결 및 인물 변화
            - 세계관에 미친 영향과 여운 남기기

        위 템플릿에 따라 소설의 줄거리를 상세하게 작성해 주세요. 
        """
        model = genai.GenerativeModel("models/gemini-2.0-flash", system_instruction=instruction)
        prompt = f"## 소설 장르: {self.genre}\n## 소설 제목: {self.title}\n## 소설 세계관: {self.worldview}\n\n**소설 줄거리**\n"
        response = model.generate_content(prompt)
        self.synopsis = response.text
        print("소설 줄거리:\n", self.synopsis)
        return self.synopsis

    def recommend_characters(self) -> str:
        """소설 등장인물 생성 함수"""
        instruction = """
        당신은 전문적으로 등장인물을 구성하는 소설 작가입니다. 
        주어진 장르, 제목, 세계관, 줄거리를 기반으로 소설 등장인물을 만들어주세요.

        각 등장인물은 다음 속성을 포함하는 JSON 형태(dict)로 표현되어야 합니다.

        * 이름: (예: 홍길동, 춘향이 등) - 등장인물의 이름 (필수)
        * 성별: (예: 남, 여, 기타) - 등장인물의 성별 (필수)
        * 나이: (예: 20세, 30대 초반 등) - 등장인물의 나이 (필수)
        * 역할: (예: 주인공, 조력자, 악당 등) - 이야기 속 역할 (필수)
        * 직업: (예: 의사, 학생, 무사 등) - 등장인물의 직업 (필수)
        * 프로필: 등장인물의 외모, 성격, 능력, 과거, 관계 등 여러 특징을 하나의 자연스러운 문장으로 작성해 주세요.
          (예: "키 180cm에 날카로운 눈매와 과묵한 성격을 가지고 있으며, 특정 능력과 습관, 버릇, 가치관 등이 돋보이고, 가문 및 출신과 과거가 있으며, 주인공과 친구 혹은 연인 관계를 형성한다.")
            - (예: 키 180cm, 날카로운 눈매, 과묵한 성격 등) - 외모, 성격, 능력 등 세부 묘사 (선택)
            - (예: 특정 능력, 습관, 버릇, 가치관 등) - 등장인물의 개성을 드러내는 특징 (선택)
            - (예: 가문, 출신, 과거 등) - 등장인물의 과거와 배경 (선택)
            - (예: 주인공과 친구, 연인 관계 등) - 다른 등장인물과의 관계 (선택)

        여러 명의 등장인물을 생성할 수 있으며, 각 등장인물은 아래와 같은 형태로 표현되어야 합니다.

        characters =
            {
                "이름": "홍길동",
                "성별": "남",
                "나이": "20",
                "역할": "주인공",
                "직업": "무사",
                "프로필": "활달한 성격"
            },
            {
                "이름": "춘향이",
                "성별": "여",
                "나이": "18",
                "역할": "조력자",
                "직업": "농부",
                "프로필": "밝고 활발한 성격"
            }
        """
        model = genai.GenerativeModel("models/gemini-2.0-flash", system_instruction=instruction)
        prompt = (
            f"## 소설 장르: {self.genre}\n"
            f"## 소설 제목: {self.title}\n"
            f"## 소설 세계관: {self.worldview}\n"
            f"## 소설 줄거리: {self.synopsis}\n"
            f"## 기존 소설 등장인물: {self.characters}\n\n"
            "**새로운 소설 등장인물**\n"
        )
        response = model.generate_content(prompt)
        new_characters = response.text
        new_characters = re.sub(r'```json\n(.*?)\n```', r'\1', new_characters, flags=re.DOTALL)
        try:
            existing = json.loads(self.characters) if self.characters.strip() else []
            if not isinstance(existing, list):
                existing = [existing]
        except Exception:
            existing = []
        try:
            new_char = json.loads(new_characters)
        except Exception:
            new_char = new_characters
        # 수정: new_char가 리스트인 경우 extend, 그렇지 않으면 append
        if isinstance(new_char, list):
            existing.extend(new_char)
        else:
            existing.append(new_char)
        self.characters = json.dumps(existing, ensure_ascii=False, indent=2)
        print("소설 등장인물:\n", self.characters)
        return self.characters

    def add_new_characters(self) -> str:
        """
        기존 등장인물에 추가로 새로운 등장인물을 생성하는 함수.
        기존 캐릭터와 차별화된 새로운 인물들을 생성하여 기존 목록 리스트에 덧붙입니다.
        """
        instruction = """
        당신은 전문적으로 등장인물을 구성하는 소설 작가입니다. 
        주어진 장르, 제목, 세계관, 줄거리, 기존 등장인물들을 기반으로 기존과 다른 새로운 소설 등장인물을 만들어주세요.

        등장인물은 다음 속성을 포함하는 JSON 형태(dict)로 표현되어야 합니다.

        * 이름: (예: 홍길동, 춘향이 등) - 등장인물의 이름 (필수)
        * 성별: (예: 남, 여, 기타) - 등장인물의 성별 (필수)
        * 나이: (예: 20세, 30대 초반 등) - 등장인물의 나이 (필수)
        * 역할: (예: 주인공, 조력자, 악당 등) - 이야기 속 역할 (필수)
        * 직업: (예: 의사, 학생, 무사 등) - 등장인물의 직업 (필수)
        * 프로필: 등장인물의 외모, 성격, 능력, 과거, 관계 등 여러 특징을 하나의 자연스러운 문장으로 작성해 주세요.
          (예: "키 180cm에 날카로운 눈매와 과묵한 성격을 가지고 있으며, 특정 능력과 습관, 버릇, 가치관 등이 돋보이고, 가문 및 출신과 과거가 있으며, 주인공과 친구 혹은 연인 관계를 형성한다.")
        등장인물은 아래와 같은 형태로 표현되어야 하며, 한 명의 캐릭터를 생성합니다.

        characters =
            {
                "이름": "홍길동",
                "성별": "남",
                "나이": "20",
                "역할": "주인공",
                "직업": "무사",
                "프로필": "활달한 성격"
            }
        """
        model = genai.GenerativeModel("models/gemini-2.0-flash", system_instruction=instruction)
        prompt = (
            f"## 소설 장르: {self.genre}\n"
            f"## 소설 제목: {self.title}\n"
            f"## 소설 세계관: {self.worldview}\n"
            f"## 소설 줄거리: {self.synopsis}\n"
            f"## 기존 소설 등장인물: {self.characters}\n\n"
            "**추가 생성된 소설 등장인물**\n"
        )
        response = model.generate_content(prompt)
        additional_characters = response.text
        additional_characters = re.sub(r'```json\n(.*?)\n```', r'\1', additional_characters, flags=re.DOTALL)
        try:
            existing = json.loads(self.characters) if self.characters.strip() else []
            if not isinstance(existing, list):
                existing = [existing]
        except Exception:
            existing = []
        try:
            new_char = json.loads(additional_characters)
        except Exception:
            new_char = additional_characters
        if isinstance(new_char, list):
            existing.extend(new_char)
        else:
            existing.append(new_char)
        self.characters = json.dumps(existing, ensure_ascii=False, indent=2)
        print("소설 등장인물 업데이트:\n", self.characters)
        return self.characters

    def create_episode(self) -> str:
        """
        DB에 저장된 이전 에피소드들을 모두 불러와서, 
        이를 기반으로 새 에피소드(초안 또는 다음 화)를 생성합니다.
        각 에피소드는 500-1000자 정도로 작성합니다.
        """
        previous_episodes = self.get_previous_episodes()
        
        if not previous_episodes:
            # 이전 에피소드가 없다면 첫 번째 에피소드(초안) 생성
            instruction = """
            당신은 탁월한 창의력과 독창성을 가진 전문 소설 작가입니다.
            아래 지시사항에 따라 소설 초안을 작성해주세요. 
            생성되는 텍스트는 순수한 일반 텍스트 형식이어야 하며, 어떠한 마크다운 문법(예: **, ## 등)도 사용하지 마세요.
            소설은 주어진 장르, 제목, 세계관, 줄거리, 등장인물의 정보를 바탕으로 작성되어야 하며, 독자에게 강렬한 첫인상을 남길 수 있도록 생생한 묘사와 몰입감 있는 서술을 포함해야 합니다.
            에피소드는 700-1500자 분량으로 작성해주세요.

            작성 시 다음 항목들을 반드시 반영합니다:
            1. 소설의 분위기와 문체를 장르에 맞게 설정합니다.
            2. 세계관, 시대, 문화 및 기술적 배경을 구체적이고 생동감 있게 설명합니다.
            3. 주인공 및 주요 인물들의 등장과 그들이 추구하는 목표, 내면의 동기를 명확히 제시합니다.
            4. 독자의 흥미를 끌 수 있도록 초반부터 미스터리나 긴장감을 암시하는 요소를 포함합니다.
            5. 텍스트의 마지막 문장은 반드시 완성된 문장으로 구성되고, 자연스러운 마무리로 작성합니다.
            """
            episode_label = "**소설 초안**\n"
        else:
            # 이전 에피소드가 있다면 DB의 모든 내용을 기반으로 다음 화 생성
            instruction = """
            당신은 탁월한 창의력과 독창성을 가진 전문 소설 작가입니다. 
            아래 지시사항에 따라 소설의 다음 화를 작성해주세요. 
            생성되는 텍스트는 순수한 일반 텍스트 형식이어야 하며, 어떠한 마크다운 문법(예: **, ## 등)도 사용하지 마세요.
            소설은 주어진 장르, 제목, 세계관, 줄거리, 등장인물 및 이전 에피소드의 내용을 기반으로 자연스럽게 이어지는 이야기를 구성해야 합니다.
            에피소드는 700-1500자 분량으로 작성해주세요.

            작성 시 다음 항목들을 반드시 반영합니다:
            1. 소설의 분위기와 문체를 장르에 맞게 유지하며, 이전 에피소드의 흐름과 일관되게 전개합니다.
            2. 이전 에피소드의 주요 사건들을 참고하여 새로운 갈등, 도전 및 전환점을 도입합니다.
            3. 인물 간의 관계, 내면적 변화 및 상호작용을 심도 있게 묘사합니다.
            4. 이야기의 전개와 함께 긴장감, 미스터리 또는 서스펜스 요소를 적절히 배치합니다.
            5. 텍스트의 마지막 문장은 반드시 완성된 문장으로 구성되고, 이야기의 흐름이 자연스럽게 마무리되도록 작성합니다.
            """
            episode_label = "**소설 다음화**\n"
        
        model = genai.GenerativeModel("models/gemini-2.0-flash", system_instruction=instruction)
        prompt = (
            f"## 소설 장르: {self.genre}\n"
            f"## 소설 제목: {self.title}\n"
            f"## 소설 세계관: {self.worldview}\n"
            f"## 소설 줄거리: {self.synopsis}\n"
            f"## 소설 등장인물: {self.characters}\n"
        )
        if previous_episodes:
            prompt += f"## 소설 이전화: {previous_episodes}\n\n"
        prompt += episode_label

        response = model.generate_content(prompt)
        episode_content = response.text

        # 에피소드 번호는 기존 에피소드 수 + 1
        cursor = self.db_connection.cursor()
        cursor.execute("SELECT COUNT(*) FROM episode WHERE novel_pk = %s", (self.novel_pk,))
        (count,) = cursor.fetchone()
        cursor.close()
        ep_number = count + 1
        ep_title = f"Episode {ep_number}"

        # 새로 생성된 에피소드를 DB에 저장
        self.insert_episode(ep_title, episode_content)

        if ep_number == 1:
            self.first_episode = episode_content
            print("에피소드 1화:\n", episode_content)
        else:
            self.next_episode = episode_content
            print(f"에피소드 {ep_number}화:\n", episode_content)
        return episode_content


# 테스트용 Class

In [ ]:
import os
import re
import json
from dotenv import load_dotenv
import google.generativeai as genai
import mysql.connector
from mysql.connector import errors

class NovelGenerator:
    def __init__(self, genre: str, title: str):
        # 환경변수 로드 및 API, DB 설정
        load_dotenv()
        self.GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
        genai.configure(api_key=self.GEMINI_API_KEY)

        self.genre = genre
        self.title = title
        self.worldview = ""
        self.synopsis = ""
        self.characters = ""
        self.first_chapter = ""
        self.next_chapter = ""
        
        # 여기서는 간단한 예시로 단일 소설에 대해 novel_pk를 1로 고정
        self.novel_pk = 1
        
        # DB 접속정보
        self.db_host = os.getenv("DB_HOST")
        self.db_port = int(os.getenv("DB_PORT", "3306"))
        self.db_user = os.getenv("DB_USER")
        self.db_password = os.getenv("DB_PASSWORD")
        self.db_name = os.getenv("DB_NAME")
        
        # 데이터베이스가 없는 경우 생성하도록 처리
        try:
            self.db_connection = mysql.connector.connect(
                host=self.db_host,
                port=self.db_port,
                user=self.db_user,
                password=self.db_password,
                database=self.db_name
            )
        except errors.ProgrammingError as e:
            if e.errno == 1049:  # Unknown database
                print(f"데이터베이스 {self.db_name}가 존재하지 않습니다. 데이터베이스를 생성합니다.")
                temp_conn = mysql.connector.connect(
                    host=self.db_host,
                    port=self.db_port,
                    user=self.db_user,
                    password=self.db_password
                )
                temp_cursor = temp_conn.cursor()
                temp_cursor.execute(f"CREATE DATABASE IF NOT EXISTS {self.db_name}")
                temp_conn.commit()
                temp_cursor.close()
                temp_conn.close()
                # 데이터베이스 생성 후 다시 연결
                self.db_connection = mysql.connector.connect(
                    host=self.db_host,
                    port=self.db_port,
                    user=self.db_user,
                    password=self.db_password,
                    database=self.db_name
                )
            else:
                raise e
        
        self.create_table_if_not_exists()

    def create_table_if_not_exists(self):
        """episode 테이블이 없으면 생성"""
        cursor = self.db_connection.cursor()
        query = """
        CREATE TABLE IF NOT EXISTS episode (
            ep_pk INT AUTO_INCREMENT PRIMARY KEY,
            ep_title VARCHAR(255) NOT NULL,
            novel_pk INT NOT NULL,
            created_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            updated_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
            ep_content TEXT NOT NULL
        )
        """
        cursor.execute(query)
        self.db_connection.commit()
        cursor.close()

    def insert_episode(self, ep_title: str, ep_content: str):
        """새 에피소드를 DB에 저장합니다."""
        cursor = self.db_connection.cursor()
        query = """
        INSERT INTO episode (ep_title, novel_pk, ep_content)
        VALUES (%s, %s, %s)
        """
        cursor.execute(query, (ep_title, self.novel_pk, ep_content))
        self.db_connection.commit()
        cursor.close()

    def get_previous_episodes(self) -> str:
        """해당 소설의 모든 에피소드(ep_content)를 created_date 기준 오름차순으로 불러와 하나의 문자열로 합칩니다."""
        cursor = self.db_connection.cursor()
        query = """
        SELECT ep_content FROM episode
        WHERE novel_pk = %s
        ORDER BY created_date ASC
        """
        cursor.execute(query, (self.novel_pk,))
        rows = cursor.fetchall()
        cursor.close()
        episodes = "\n\n---\n\n".join([row[0] for row in rows])
        return episodes

    def recommend_worldview(self) -> str:
        """소설 세계관 생성 함수"""
        instruction = """
        당신은 전문적으로 소설 세계관을 만드는 작가입니다.
        생성되는 텍스트는 순수한 일반 텍스트 형식이어야 하며, 어떠한 마크다운 문법(예: **, ## 등)도 사용하지 말아주세요.
        주어진 장르, 제목을 기반으로 독창적이고 생동감 있는 소설 세계관을 만들어주세요.
        소설 세계관을 구성할 때, 아래의 항목들을 참고하여 상세하게 작성하세요.

        1. 기본 설정
            - 시대: (예: 과거, 현대, 미래 등) - 시대적 배경과 분위기를 구체적으로 설명해주세요.
            - 핵심 테마: (예: 인간과 자연의 대립, 기술과 마법의 공존 등) - 세계관을 관통하는 핵심 주제를 명확히 제시하고, 다른 요소들과의 연관성을 설명해주세요.

        2. 지리적 배경
            - 세계 전체의 구조(대륙, 국가, 도시, 마을 등) - 지리적 구조를 시각적으로 표현하고, 각 지역의 특징을 설명해주세요.
            - 주요 지형 및 자연 환경(기후, 산맥, 강 등) - 자연 환경이 세계관에 미치는 영향을 설명하고, 상징적인 의미를 부여할 수 있습니다.
            - 상징적이거나 중요한 장소(예: 전설적인 성, 신비로운 숲 등) - 각 장소의 역사, 특징, 중요성 등을 구체적으로 설명해주세요.

        3. 역사와 문화
            - 세계의 건국 신화 및 주요 역사적 사건 - 역사적 사건이 세계관에 미친 영향과 현재 사회에 남은 유산을 설명해주세요.
            - 사회 구조(정치 체제, 경제 시스템, 계층 구조 등) - 사회 구조가 세계관 구성원들의 삶에 미치는 영향을 설명해주세요.
            - 문화와 종교(전통, 의식, 종교적 신념 등) - 문화와 종교가 세계관에 미치는 영향을 설명하고, 독특한 요소들을 강조해주세요.

        4. 기술 및 초자연적 요소
            - 세계 내 기술 수준(과거, 현대, 미래 기술) - 기술 수준이 세계관에 미치는 영향과 특징을 설명해주세요.
            - 만약 존재한다면 마법이나 초자연적 현상의 규칙과 한계 - 마법/초자연 현상이 세계관에 미치는 영향과 작동 방식을 구체적으로 설명해주세요.

        5. 인물 및 종족
            - 주요 인물이나 집단, 종족의 특징과 역할 - 각 인물/종족의 개성과 세계관 내 역할을 명확하게 설명해주세요.
            - 각 인물/종족이 속한 사회적, 문화적 배경 - 인물/종족의 배경이 그들의 행동 양식에 미치는 영향을 설명해주세요.

        6. 갈등 구조
            - 사회 내부 혹은 외부의 갈등 요소(정치적 분쟁, 종족 간 충돌 등) - 갈등의 원인, 진행 과정, 결과를 구체적으로 설명해주세요.
            - 이러한 갈등이 전체 세계관에 미치는 영향 - 갈등이 세계관의 다른 요소들에 미치는 영향을 설명해주세요.

        7. 세부 설정 및 고유 용어
            - 세계관 내에서만 사용되는 독특한 용어나 법칙 정리 - 용어의 의미와 사용 예시를 설명하여 세계관의 현실성을 높여주세요.
            - 설정의 일관성을 유지할 수 있도록 참고 자료 작성 - 필요한 경우, 추가적인 자료를 작성하여 세계관의 완성도를 높여주세요.
        """
        model = genai.GenerativeModel("models/gemini-2.0-flash", system_instruction=instruction)
        prompt = f"## 소설 장르: {self.genre}\n## 소설 제목: {self.title}\n\n**소설 세계관**\n"
        response = model.generate_content(prompt)
        self.worldview = response.text
        print("소설 세계관:\n", self.worldview)
        return self.worldview

    def recommend_synopsis(self) -> str:
        """소설 줄거리 생성 함수"""
        instruction = """
        당신은 전문적으로 소설 줄거리를 만드는 작가입니다.
        생성되는 텍스트는 순수한 일반 텍스트 형식이어야 하며, 어떠한 마크다운 문법(예: **, ## 등)도 사용하지 말아주세요.
        주어진 장르, 제목, 세계관을 기반으로 독창적이고 생동감 있는 소설 줄거리를 만들어주세요.
        소설 줄거리를 구성할 때, 아래의 항목들을 참고하세요.

        1. 기본 정보
            - 장르: [예: 판타지, SF 등]
            - 제목: [제목 입력]
            - 세계관: [세계관의 배경, 시대, 문화, 기술 등 간단히 설명]
        
        2. 발단
            - 소설이 펼쳐지는 세계관의 기초 소개
            - 주요 인물들의 소개 및 그들이 추구하는 기본 목표
            - 이야기에 숨은 첫 갈등이나 문제의 암시
        
        3. 전개
            - 주된 갈등 및 도전 과제의 본격적인 전개
            - 인물들 간의 관계, 서브 플롯, 그리고 각 인물의 내면적 변화
            - 주인공이 맞서야 하는 문제와 성장 과정을 상세히 설명
        
        4. 위기
            - 갈등이 극한으로 치닫는 순간, 상황이 악화되고 긴장이 고조됨
            - 주인공과 주변 인물들이 맞닥뜨리는 결정적 어려움 및 시련
            - 이야기의 전환점 역할을 하는 중요한 사건 발생
        
        5. 절정
            - 갈등의 최고조, 결정적인 대립과 순간의 전환
            - 주인공의 중대한 선택 및 운명을 건 마지막 대결
            - 전체 이야기의 긴장감과 감정이 폭발하는 순간
        
        6. 결말
            - 갈등의 해소와 함께 인물들의 변화 및 성장 결과 제시
            - 사건이 세계관에 미친 영향과 이후의 여운 남기는 마무리
            - 독자에게 남기는 메시지 혹은 후일담

        위 템플릿에 따라 소설의 줄거리를 상세하게 작성해 주세요. 
        """
        model = genai.GenerativeModel("models/gemini-2.0-flash", system_instruction=instruction)
        prompt = f"## 소설 장르: {self.genre}\n## 소설 제목: {self.title}\n## 소설 세계관: {self.worldview}\n\n**소설 줄거리**\n"
        response = model.generate_content(prompt)
        self.synopsis = response.text
        print("소설 줄거리:\n", self.synopsis)
        return self.synopsis

    def recommend_characters(self) -> str:
        """소설 등장인물 생성 함수"""
        instruction = """
        당신은 전문적으로 등장인물을 구성하는 소설 작가입니다. 
        주어진 장르, 제목, 세계관, 줄거리를 기반으로 소설 등장인물을 만들어주세요.

        각 등장인물은 다음 속성을 포함하는 JSON 형태(dict)로 표현되어야 합니다.

        * 이름: (예: 홍길동, 춘향이 등) - 등장인물의 이름 (필수)
        * 성별: (예: 남, 여, 기타) - 등장인물의 성별 (필수)
        * 나이: (예: 20세, 30대 초반 등) - 등장인물의 나이 (필수)
        * 역할: (예: 주인공, 조력자, 악당 등) - 이야기 속 역할 (필수)
        * 직업: (예: 의사, 학생, 무사 등) - 등장인물의 직업 (필수)
        * 프로필: 등장인물의 외모, 성격, 능력, 과거, 관계 등 여러 특징을 하나의 자연스러운 문장으로 작성해 주세요.
          (예: "키 180cm에 날카로운 눈매와 과묵한 성격을 가지고 있으며, 특정 능력과 습관, 버릇, 가치관 등이 돋보이고, 가문 및 출신과 과거가 있으며, 주인공과 친구 혹은 연인 관계를 형성한다.")
            - (예: 키 180cm, 날카로운 눈매, 과묵한 성격 등) - 외모, 성격, 능력 등 세부 묘사 (선택)
            - (예: 특정 능력, 습관, 버릇, 가치관 등) - 등장인물의 개성을 드러내는 특징 (선택)
            - (예: 가문, 출신, 과거 등) - 등장인물의 과거와 배경 (선택)
            - (예: 주인공과 친구, 연인 관계 등) - 다른 등장인물과의 관계 (선택)

        여러 명의 등장인물을 생성할 수 있으며, 각 등장인물은 아래와 같은 형태로 표현되어야 합니다.

        characters =
            {
                "이름": "홍길동",
                "성별": "남",
                "나이": "20",
                "역할": "주인공",
                "직업": "무사",
                "프로필": "활달한 성격"
            },
            {
                "이름": "춘향이",
                "성별": "여",
                "나이": "18",
                "역할": "조력자",
                "직업": "농부",
                "프로필": "밝고 활발한 성격"
            }
        """
        model = genai.GenerativeModel("models/gemini-2.0-flash", system_instruction=instruction)
        prompt = (
            f"## 소설 장르: {self.genre}\n"
            f"## 소설 제목: {self.title}\n"
            f"## 소설 세계관: {self.worldview}\n"
            f"## 소설 줄거리: {self.synopsis}\n"
            f"## 기존 소설 등장인물: {self.characters}\n\n"
            "**새로운 소설 등장인물**\n"
        )
        response = model.generate_content(prompt)
        new_characters = response.text
        new_characters = re.sub(r'```json\n(.*?)\n```', r'\1', new_characters, flags=re.DOTALL)
        try:
            existing = json.loads(self.characters) if self.characters.strip() else []
            if not isinstance(existing, list):
                existing = [existing]
        except Exception:
            existing = []
        try:
            new_char = json.loads(new_characters)
        except Exception:
            new_char = new_characters
        # 수정: new_char가 리스트인 경우 extend, 그렇지 않으면 append
        if isinstance(new_char, list):
            existing.extend(new_char)
        else:
            existing.append(new_char)
        self.characters = json.dumps(existing, ensure_ascii=False, indent=2)
        print("소설 등장인물:\n", self.characters)
        return self.characters

    def add_new_characters(self) -> str:
        """
        기존 등장인물에 추가로 새로운 등장인물을 생성하는 함수.
        기존 캐릭터와 차별화된 새로운 인물들을 생성하여 기존 목록 리스트에 덧붙입니다.
        """
        instruction = """
        당신은 전문적으로 등장인물을 구성하는 소설 작가입니다. 
        주어진 장르, 제목, 세계관, 줄거리, 기존 등장인물들을 기반으로 기존과 다른 새로운 소설 등장인물을 만들어주세요.

        등장인물은 다음 속성을 포함하는 JSON 형태(dict)로 표현되어야 합니다.

        * 이름: (예: 홍길동, 춘향이 등) - 등장인물의 이름 (필수)
        * 성별: (예: 남, 여, 기타) - 등장인물의 성별 (필수)
        * 나이: (예: 20세, 30대 초반 등) - 등장인물의 나이 (필수)
        * 역할: (예: 주인공, 조력자, 악당 등) - 이야기 속 역할 (필수)
        * 직업: (예: 의사, 학생, 무사 등) - 등장인물의 직업 (필수)
        * 프로필: 등장인물의 외모, 성격, 능력, 과거, 관계 등 여러 특징을 하나의 자연스러운 문장으로 작성해 주세요.
          (예: "키 180cm에 날카로운 눈매와 과묵한 성격을 가지고 있으며, 특정 능력과 습관, 버릇, 가치관 등이 돋보이고, 가문 및 출신과 과거가 있으며, 주인공과 친구 혹은 연인 관계를 형성한다.")
        등장인물은 아래와 같은 형태로 표현되어야 하며, 한 명의 캐릭터를 생성합니다.

        characters =
            {
                "이름": "홍길동",
                "성별": "남",
                "나이": "20",
                "역할": "주인공",
                "직업": "무사",
                "프로필": "활달한 성격"
            }
        """
        model = genai.GenerativeModel("models/gemini-2.0-flash", system_instruction=instruction)
        prompt = (
            f"## 소설 장르: {self.genre}\n"
            f"## 소설 제목: {self.title}\n"
            f"## 소설 세계관: {self.worldview}\n"
            f"## 소설 줄거리: {self.synopsis}\n"
            f"## 기존 소설 등장인물: {self.characters}\n\n"
            "**추가 생성된 소설 등장인물**\n"
        )
        response = model.generate_content(prompt)
        additional_characters = response.text
        additional_characters = re.sub(r'```json\n(.*?)\n```', r'\1', additional_characters, flags=re.DOTALL)
        try:
            existing = json.loads(self.characters) if self.characters.strip() else []
            if not isinstance(existing, list):
                existing = [existing]
        except Exception:
            existing = []
        try:
            new_char = json.loads(additional_characters)
        except Exception:
            new_char = additional_characters
        if isinstance(new_char, list):
            existing.extend(new_char)
        else:
            existing.append(new_char)
        self.characters = json.dumps(existing, ensure_ascii=False, indent=2)
        print("소설 등장인물 업데이트:\n", self.characters)
        return self.characters

    def create_episode(self) -> str:
        """
        DB에 저장된 이전 에피소드들을 모두 불러와서, 
        이를 기반으로 새 에피소드(초안 또는 다음 화)를 생성합니다.
        각 에피소드는 500-1000자 정도로 작성합니다.
        """
        previous_episodes = self.get_previous_episodes()
        
        if not previous_episodes:
            # 이전 에피소드가 없다면 첫 번째 에피소드(초안) 생성
            instruction = """
            당신은 탁월한 창의력과 독창성을 가진 전문 소설 작가입니다.
            아래 지시사항에 따라 소설 초안을 작성해주세요. 
            생성되는 텍스트는 순수한 일반 텍스트 형식이어야 하며, 어떠한 마크다운 문법(예: **, ## 등)도 사용하지 마세요.
            주어진 장르, 제목, 세계관, 줄거리, 등장인물의 정보를 바탕으로 에피소드 초안이 작성되어야 합니다.
            에피소드는 700-1500자 분량으로 작성하세요.

            작성 시 다음 항목들을 반드시 반영합니다:
            1. 소설의 분위기와 문체를 장르에 맞게 설정합니다.
            2. 텍스트의 마지막 문장은 반드시 완성된 문장으로 구성되고, 자연스러운 마무리로 작성합니다.
            """
            episode_label = "**소설 초안**\n"
        else:
            # 이전 에피소드가 있다면 DB의 모든 내용을 기반으로 다음 화 생성
            instruction = """
            당신은 탁월한 창의력과 독창성을 가진 전문 소설 작가입니다. 
            아래 지시사항에 따라 소설의 다음 화를 작성해주세요. 
            생성되는 텍스트는 순수한 일반 텍스트 형식이어야 하며, 어떠한 마크다운 문법(예: **, ## 등)도 사용하지 마세요.
            주어진 장르, 제목, 세계관, 줄거리, 등장인물 및 이전 에피소드의 내용을 기반으로 다음 에피소드를 작성해야 합니다.
            에피소드는 700-1500자 분량으로 작성하세요.

            작성 시 다음 항목들을 반드시 반영합니다:
            1. 소설의 분위기와 문체를 장르에 맞게 유지하며, 이전 에피소드의 흐름과 일관되게 전개합니다.
            2. 이전 에피소드의 내용을 참고하여 내용이 자연스럽게 연결되도록 작성하세요.
            3. 텍스트의 마지막 문장은 반드시 완성된 문장으로 구성되고, 이야기의 흐름이 자연스럽게 마무리되도록 작성합니다.
            """
            episode_label = "**소설 다음화**\n"
        
        model = genai.GenerativeModel("models/gemini-2.0-flash", system_instruction=instruction)
        prompt = (
            f"## 소설 장르: {self.genre}\n"
            f"## 소설 제목: {self.title}\n"
            f"## 소설 세계관: {self.worldview}\n"
            f"## 소설 줄거리: {self.synopsis}\n"
            f"## 소설 등장인물: {self.characters}\n"
        )
        if previous_episodes:
            prompt += f"## 소설 이전 에피소드: {previous_episodes}\n\n"
        prompt += episode_label

        response = model.generate_content(prompt)
        episode_content = response.text

        # 에피소드 번호는 기존 에피소드 수 + 1
        cursor = self.db_connection.cursor()
        cursor.execute("SELECT COUNT(*) FROM episode WHERE novel_pk = %s", (self.novel_pk,))
        (count,) = cursor.fetchone()
        cursor.close()
        ep_number = count + 1
        ep_title = f"Episode {ep_number}"

        # 새로 생성된 에피소드를 DB에 저장
        self.insert_episode(ep_title, episode_content)

        if ep_number == 1:
            self.first_episode = episode_content
            print("에피소드 1화:\n", episode_content)
        else:
            self.next_episode = episode_content
            print(f"에피소드 {ep_number}화:\n", episode_content)
        return episode_content


## 장르, 제목 입력

In [87]:
# 소설의 장르와 제목을 입력합니다.
genre = input("소설의 장르를 입력하세요: ex) 판타지, 로맨스, 스릴러 등\n")
title = input("소설의 제목을 입력하세요: ex) 괴식식당, 신의탑, 전생전기 등\n")
print(genre)
print(title)

novel_gen = NovelGenerator(genre, title)

미스터리, 스릴러
연필살인마
데이터베이스 novel_test_db가 존재하지 않습니다. 데이터베이스를 생성합니다.


## 세계관 생성

In [88]:
novel_gen.recommend_worldview()

소설 세계관:
 1. 기본 설정

시대: 현대, 2024년 대한민국 서울. 고도로 발전된 도시의 화려함 뒤에 숨겨진 어두운 그림자를 배경으로 한다. 정보화 시대의 익명성과 감시 사회의 억압이 공존하는 불균형적인 사회 분위기를 띤다.

핵심 테마: '익명성과 책임의식의 부재'. 인터넷이라는 익명의 공간에서 벌어지는 악플, 사이버 폭력 등의 문제가 현실 세계의 살인사건으로 이어진다는 설정을 통해 현대 사회의 익명성이 개인에게 미치는 영향과 책임의식의 중요성을 부각한다. 또한, 기술 발전에 따른 감시 사회의 윤리적 문제점을 제기하며, 진실 추적 과정에서 드러나는 인간의 어두운 본성을 탐구한다.

2. 지리적 배경

세계 전체의 구조: 이야기는 대한민국 서울, 그중에서도 강남의 화려한 오피스 지구와 낙후된 구도심 지역을 중심으로 전개된다. 강남은 부와 권력의 상징으로, 사건의 배경이 되는 주요 장소다. 구도심 지역은 사회적 불평등과 소외된 계층의 삶을 보여주며, 사건의 중요한 단서가 숨겨진 장소로 등장한다.

주요 지형 및 자연 환경: 서울의 빽빽한 고층 빌딩 숲과 복잡한 지하철 노선은 익명성이 보장되는 현대 사회의 특징을 상징한다. 한강은 도시를 가로지르며 사건의 중요한 모티브가 되는 장소로, 희생자들의 고독과 절망을 반영한다. 미세먼지로 뒤덮인 하늘은 답답하고 불확실한 미래를 암시한다.

상징적이거나 중요한 장소:
- 강남의 대형 IT 기업: 악플 사건의 진원지로, 익명성에 숨어 타인을 공격하는 현대인의 그림자를 상징한다.
- 낡은 구도심의 PC방: 용의자가 은신하거나 범행 계획을 세우는 장소로, 사회에서 소외된 사람들의 분노와 좌절감을 보여준다.
- 한강변 공원: 희생자들이 마지막으로 목격된 장소이자, 사건 해결의 중요한 단서가 발견되는 곳으로, 삶과 죽음의 경계를 상징한다.

3. 역사와 문화

세계의 건국 신화 및 주요 역사적 사건: 한국 근현대사의 주요 사건들은 직접적으로 언급되지는 않지만, 사회 전반에 걸쳐 뿌리 깊게 자리 잡은 권위주의, 집단주의, 경쟁

"1. 기본 설정\n\n시대: 현대, 2024년 대한민국 서울. 고도로 발전된 도시의 화려함 뒤에 숨겨진 어두운 그림자를 배경으로 한다. 정보화 시대의 익명성과 감시 사회의 억압이 공존하는 불균형적인 사회 분위기를 띤다.\n\n핵심 테마: '익명성과 책임의식의 부재'. 인터넷이라는 익명의 공간에서 벌어지는 악플, 사이버 폭력 등의 문제가 현실 세계의 살인사건으로 이어진다는 설정을 통해 현대 사회의 익명성이 개인에게 미치는 영향과 책임의식의 중요성을 부각한다. 또한, 기술 발전에 따른 감시 사회의 윤리적 문제점을 제기하며, 진실 추적 과정에서 드러나는 인간의 어두운 본성을 탐구한다.\n\n2. 지리적 배경\n\n세계 전체의 구조: 이야기는 대한민국 서울, 그중에서도 강남의 화려한 오피스 지구와 낙후된 구도심 지역을 중심으로 전개된다. 강남은 부와 권력의 상징으로, 사건의 배경이 되는 주요 장소다. 구도심 지역은 사회적 불평등과 소외된 계층의 삶을 보여주며, 사건의 중요한 단서가 숨겨진 장소로 등장한다.\n\n주요 지형 및 자연 환경: 서울의 빽빽한 고층 빌딩 숲과 복잡한 지하철 노선은 익명성이 보장되는 현대 사회의 특징을 상징한다. 한강은 도시를 가로지르며 사건의 중요한 모티브가 되는 장소로, 희생자들의 고독과 절망을 반영한다. 미세먼지로 뒤덮인 하늘은 답답하고 불확실한 미래를 암시한다.\n\n상징적이거나 중요한 장소:\n- 강남의 대형 IT 기업: 악플 사건의 진원지로, 익명성에 숨어 타인을 공격하는 현대인의 그림자를 상징한다.\n- 낡은 구도심의 PC방: 용의자가 은신하거나 범행 계획을 세우는 장소로, 사회에서 소외된 사람들의 분노와 좌절감을 보여준다.\n- 한강변 공원: 희생자들이 마지막으로 목격된 장소이자, 사건 해결의 중요한 단서가 발견되는 곳으로, 삶과 죽음의 경계를 상징한다.\n\n3. 역사와 문화\n\n세계의 건국 신화 및 주요 역사적 사건: 한국 근현대사의 주요 사건들은 직접적으로 언급되지는 않지만, 사회 전반에 걸쳐 뿌리 깊게 자리 잡은 권위주

## 줄거리 생성

In [90]:
novel_gen.recommend_synopsis()

소설 줄거리:
 1. 기본 정보
    - 장르: 미스터리, 스릴러
    - 제목: 연필살인마
    - 세계관: 2024년 대한민국 서울. 고도로 발전된 도시의 화려함 뒤에 숨겨진 어두운 그림자를 배경으로 한다. 정보화 시대의 익명성과 감시 사회의 억압이 공존하는 불균형적인 사회 분위기를 띤다. 인터넷이라는 익명의 공간에서 벌어지는 악플, 사이버 폭력 등의 문제가 현실 세계의 살인사건으로 이어진다는 설정을 통해 현대 사회의 익명성이 개인에게 미치는 영향과 책임의식의 중요성을 부각한다.

2. 발단
    - 서울 강남, 유명 IT 기업의 신입사원이 한강에서 변사체로 발견된다. 사건을 담당하게 된 베테랑 형사 강민준은 단순 자살로 위장된 정황 속에서 타살의 냄새를 맡는다. 피해자는 생전 온라인 커뮤니티에서 악플에 시달렸으며, 사망 직전까지 극심한 정신적 고통을 겪었다는 사실이 드러난다. 강민준은 피해자를 괴롭힌 악플러들을 수사망에 올리지만, 용의자들은 완벽하게 익명 뒤에 숨어 증거를 찾기 어렵다. 한편, 비슷한 수법의 살인사건이 연이어 발생하고, 피해자들은 모두 온라인에서 악플로 고통받던 사람들이라는 공통점이 발견된다. 강민준은 이 사건이 단순한 모방범죄가 아닌, '연필살인마'라 불리는 악질적인 연쇄살인범의 소행임을 직감한다.

3. 전개
    - 강민준은 과거의 트라우마를 극복하며 사건에 몰두한다. 그는 뛰어난 해킹 실력을 가진 조력자 박수연의 도움을 받아 악플러들의 IP 주소를 추적하고, 디지털 포렌식을 통해 익명 게시글 작성자를 찾아 나선다. 수사 과정에서 강민준은 피해자들이 속한 온라인 커뮤니티의 폐쇄성과 악플 문화의 심각성을 깨닫게 된다. 악플러들은 자신들의 행위를 '정의 구현'이라 포장하며 죄책감 없이 타인을 공격하고, 익명이라는 가면 뒤에 숨어 책임을 회피한다. 강민준은 악플러들의 심리를 파악하고, 그들의 다음 범행을 막기 위해 고군분투한다. 한편, 박수연은 과거 자신도 악플 피해자였다는 사실을 고백하며, 강민준에게 연필살인마를 잡는 데 모든 

"1. 기본 정보\n    - 장르: 미스터리, 스릴러\n    - 제목: 연필살인마\n    - 세계관: 2024년 대한민국 서울. 고도로 발전된 도시의 화려함 뒤에 숨겨진 어두운 그림자를 배경으로 한다. 정보화 시대의 익명성과 감시 사회의 억압이 공존하는 불균형적인 사회 분위기를 띤다. 인터넷이라는 익명의 공간에서 벌어지는 악플, 사이버 폭력 등의 문제가 현실 세계의 살인사건으로 이어진다는 설정을 통해 현대 사회의 익명성이 개인에게 미치는 영향과 책임의식의 중요성을 부각한다.\n\n2. 발단\n    - 서울 강남, 유명 IT 기업의 신입사원이 한강에서 변사체로 발견된다. 사건을 담당하게 된 베테랑 형사 강민준은 단순 자살로 위장된 정황 속에서 타살의 냄새를 맡는다. 피해자는 생전 온라인 커뮤니티에서 악플에 시달렸으며, 사망 직전까지 극심한 정신적 고통을 겪었다는 사실이 드러난다. 강민준은 피해자를 괴롭힌 악플러들을 수사망에 올리지만, 용의자들은 완벽하게 익명 뒤에 숨어 증거를 찾기 어렵다. 한편, 비슷한 수법의 살인사건이 연이어 발생하고, 피해자들은 모두 온라인에서 악플로 고통받던 사람들이라는 공통점이 발견된다. 강민준은 이 사건이 단순한 모방범죄가 아닌, '연필살인마'라 불리는 악질적인 연쇄살인범의 소행임을 직감한다.\n\n3. 전개\n    - 강민준은 과거의 트라우마를 극복하며 사건에 몰두한다. 그는 뛰어난 해킹 실력을 가진 조력자 박수연의 도움을 받아 악플러들의 IP 주소를 추적하고, 디지털 포렌식을 통해 익명 게시글 작성자를 찾아 나선다. 수사 과정에서 강민준은 피해자들이 속한 온라인 커뮤니티의 폐쇄성과 악플 문화의 심각성을 깨닫게 된다. 악플러들은 자신들의 행위를 '정의 구현'이라 포장하며 죄책감 없이 타인을 공격하고, 익명이라는 가면 뒤에 숨어 책임을 회피한다. 강민준은 악플러들의 심리를 파악하고, 그들의 다음 범행을 막기 위해 고군분투한다. 한편, 박수연은 과거 자신도 악플 피해자였다는 사실을 고백하며, 강민준에게 연필살인마를 잡는 데 모든

## 등장인물 생성

In [91]:
novel_gen.recommend_characters()

소설 등장인물:
 [
  {
    "이름": "강민준",
    "성별": "남",
    "나이": "30대 후반",
    "역할": "주인공",
    "직업": "베테랑 형사",
    "프로필": "과거 강력 사건 트라우마를 극복하며 '연필살인마'를 쫓는 냉철하고 직감 있는 형사로, 날카로운 눈빛과 굳게 다문 입술에서 강인함이 느껴지며, 정의감과 책임감이 강하지만 내면에는 깊은 상처를 안고 있다. 동료들과의 관계는 다소 거칠지만, 속으로는 그들을 아끼며, 사건 해결을 위해서는 물불 가리지 않는 열정을 보인다."
  },
  {
    "이름": "박수연",
    "성별": "여",
    "나이": "20대 중반",
    "역할": "조력자",
    "직업": "해커",
    "프로필": "뛰어난 해킹 실력으로 강민준을 돕는 조력자로, 과거 악플 피해 경험이 있으며, 앳된 외모와 달리 차분하고 냉철한 성격을 지녔다. 정의감과 연대 의식을 느끼며 사건 해결에 적극적으로 참여하지만, 세상에 대한 불신과 냉소적인 면모도 보인다. 강민준과는 서로의 상처를 보듬어주는 관계로 발전하며, 사건 해결에 중요한 역할을 수행한다."
  },
  {
    "이름": "최성진",
    "성별": "남",
    "나이": "불명 (20대 후반 추정)",
    "역할": "악당 (연필살인마)",
    "직업": "프리랜서 프로그래머 (추정)",
    "프로필": "온라인 커뮤니티에서 악플을 주도하며 피해자들을 죽음으로 몰아가는 '연필살인마'로, 평범한 외모 뒤에 감춰진 분노와 좌절감이 느껴지며, 익명성에 숨어 타인을 조종하고 파괴하는 데 희열을 느낀다. 과거 악플 피해 경험으로 인해 사회에 대한 깊은 원한을 품고 있으며, 자신의 행위를 '정의 구현'이라고 믿는 왜곡된 신념을 가지고 있다. 온라인 게임 실력이 뛰어나며, 디지털 기기를 능숙하게 다룬다."
  },
  {
    "이름": "김지훈",
    "성별": "남",
    "나이": "20대 초반"

'[\n  {\n    "이름": "강민준",\n    "성별": "남",\n    "나이": "30대 후반",\n    "역할": "주인공",\n    "직업": "베테랑 형사",\n    "프로필": "과거 강력 사건 트라우마를 극복하며 \'연필살인마\'를 쫓는 냉철하고 직감 있는 형사로, 날카로운 눈빛과 굳게 다문 입술에서 강인함이 느껴지며, 정의감과 책임감이 강하지만 내면에는 깊은 상처를 안고 있다. 동료들과의 관계는 다소 거칠지만, 속으로는 그들을 아끼며, 사건 해결을 위해서는 물불 가리지 않는 열정을 보인다."\n  },\n  {\n    "이름": "박수연",\n    "성별": "여",\n    "나이": "20대 중반",\n    "역할": "조력자",\n    "직업": "해커",\n    "프로필": "뛰어난 해킹 실력으로 강민준을 돕는 조력자로, 과거 악플 피해 경험이 있으며, 앳된 외모와 달리 차분하고 냉철한 성격을 지녔다. 정의감과 연대 의식을 느끼며 사건 해결에 적극적으로 참여하지만, 세상에 대한 불신과 냉소적인 면모도 보인다. 강민준과는 서로의 상처를 보듬어주는 관계로 발전하며, 사건 해결에 중요한 역할을 수행한다."\n  },\n  {\n    "이름": "최성진",\n    "성별": "남",\n    "나이": "불명 (20대 후반 추정)",\n    "역할": "악당 (연필살인마)",\n    "직업": "프리랜서 프로그래머 (추정)",\n    "프로필": "온라인 커뮤니티에서 악플을 주도하며 피해자들을 죽음으로 몰아가는 \'연필살인마\'로, 평범한 외모 뒤에 감춰진 분노와 좌절감이 느껴지며, 익명성에 숨어 타인을 조종하고 파괴하는 데 희열을 느낀다. 과거 악플 피해 경험으로 인해 사회에 대한 깊은 원한을 품고 있으며, 자신의 행위를 \'정의 구현\'이라고 믿는 왜곡된 신념을 가지고 있다. 온라인 게임 실력이 뛰어나며, 디지털 기기를 능숙하게 다룬다."\n  },\n  {\n    "이름": "김지훈",\n    "성별":

## 추가 등장인물 생성

In [92]:
# 추가 등장인물 생성을 위한 기능 호출
novel_gen.add_new_characters()

소설 등장인물 업데이트:
 [
  {
    "이름": "강민준",
    "성별": "남",
    "나이": "30대 후반",
    "역할": "주인공",
    "직업": "베테랑 형사",
    "프로필": "과거 강력 사건 트라우마를 극복하며 '연필살인마'를 쫓는 냉철하고 직감 있는 형사로, 날카로운 눈빛과 굳게 다문 입술에서 강인함이 느껴지며, 정의감과 책임감이 강하지만 내면에는 깊은 상처를 안고 있다. 동료들과의 관계는 다소 거칠지만, 속으로는 그들을 아끼며, 사건 해결을 위해서는 물불 가리지 않는 열정을 보인다."
  },
  {
    "이름": "박수연",
    "성별": "여",
    "나이": "20대 중반",
    "역할": "조력자",
    "직업": "해커",
    "프로필": "뛰어난 해킹 실력으로 강민준을 돕는 조력자로, 과거 악플 피해 경험이 있으며, 앳된 외모와 달리 차분하고 냉철한 성격을 지녔다. 정의감과 연대 의식을 느끼며 사건 해결에 적극적으로 참여하지만, 세상에 대한 불신과 냉소적인 면모도 보인다. 강민준과는 서로의 상처를 보듬어주는 관계로 발전하며, 사건 해결에 중요한 역할을 수행한다."
  },
  {
    "이름": "최성진",
    "성별": "남",
    "나이": "불명 (20대 후반 추정)",
    "역할": "악당 (연필살인마)",
    "직업": "프리랜서 프로그래머 (추정)",
    "프로필": "온라인 커뮤니티에서 악플을 주도하며 피해자들을 죽음으로 몰아가는 '연필살인마'로, 평범한 외모 뒤에 감춰진 분노와 좌절감이 느껴지며, 익명성에 숨어 타인을 조종하고 파괴하는 데 희열을 느낀다. 과거 악플 피해 경험으로 인해 사회에 대한 깊은 원한을 품고 있으며, 자신의 행위를 '정의 구현'이라고 믿는 왜곡된 신념을 가지고 있다. 온라인 게임 실력이 뛰어나며, 디지털 기기를 능숙하게 다룬다."
  },
  {
    "이름": "김지훈",
    "성별": "남",
    "나이": "20

'[\n  {\n    "이름": "강민준",\n    "성별": "남",\n    "나이": "30대 후반",\n    "역할": "주인공",\n    "직업": "베테랑 형사",\n    "프로필": "과거 강력 사건 트라우마를 극복하며 \'연필살인마\'를 쫓는 냉철하고 직감 있는 형사로, 날카로운 눈빛과 굳게 다문 입술에서 강인함이 느껴지며, 정의감과 책임감이 강하지만 내면에는 깊은 상처를 안고 있다. 동료들과의 관계는 다소 거칠지만, 속으로는 그들을 아끼며, 사건 해결을 위해서는 물불 가리지 않는 열정을 보인다."\n  },\n  {\n    "이름": "박수연",\n    "성별": "여",\n    "나이": "20대 중반",\n    "역할": "조력자",\n    "직업": "해커",\n    "프로필": "뛰어난 해킹 실력으로 강민준을 돕는 조력자로, 과거 악플 피해 경험이 있으며, 앳된 외모와 달리 차분하고 냉철한 성격을 지녔다. 정의감과 연대 의식을 느끼며 사건 해결에 적극적으로 참여하지만, 세상에 대한 불신과 냉소적인 면모도 보인다. 강민준과는 서로의 상처를 보듬어주는 관계로 발전하며, 사건 해결에 중요한 역할을 수행한다."\n  },\n  {\n    "이름": "최성진",\n    "성별": "남",\n    "나이": "불명 (20대 후반 추정)",\n    "역할": "악당 (연필살인마)",\n    "직업": "프리랜서 프로그래머 (추정)",\n    "프로필": "온라인 커뮤니티에서 악플을 주도하며 피해자들을 죽음으로 몰아가는 \'연필살인마\'로, 평범한 외모 뒤에 감춰진 분노와 좌절감이 느껴지며, 익명성에 숨어 타인을 조종하고 파괴하는 데 희열을 느낀다. 과거 악플 피해 경험으로 인해 사회에 대한 깊은 원한을 품고 있으며, 자신의 행위를 \'정의 구현\'이라고 믿는 왜곡된 신념을 가지고 있다. 온라인 게임 실력이 뛰어나며, 디지털 기기를 능숙하게 다룬다."\n  },\n  {\n    "이름": "김지훈",\n    "성별":

In [93]:
# 추가 등장인물 생성을 위한 기능 호출
novel_gen.add_new_characters()

소설 등장인물 업데이트:
 [
  {
    "이름": "강민준",
    "성별": "남",
    "나이": "30대 후반",
    "역할": "주인공",
    "직업": "베테랑 형사",
    "프로필": "과거 강력 사건 트라우마를 극복하며 '연필살인마'를 쫓는 냉철하고 직감 있는 형사로, 날카로운 눈빛과 굳게 다문 입술에서 강인함이 느껴지며, 정의감과 책임감이 강하지만 내면에는 깊은 상처를 안고 있다. 동료들과의 관계는 다소 거칠지만, 속으로는 그들을 아끼며, 사건 해결을 위해서는 물불 가리지 않는 열정을 보인다."
  },
  {
    "이름": "박수연",
    "성별": "여",
    "나이": "20대 중반",
    "역할": "조력자",
    "직업": "해커",
    "프로필": "뛰어난 해킹 실력으로 강민준을 돕는 조력자로, 과거 악플 피해 경험이 있으며, 앳된 외모와 달리 차분하고 냉철한 성격을 지녔다. 정의감과 연대 의식을 느끼며 사건 해결에 적극적으로 참여하지만, 세상에 대한 불신과 냉소적인 면모도 보인다. 강민준과는 서로의 상처를 보듬어주는 관계로 발전하며, 사건 해결에 중요한 역할을 수행한다."
  },
  {
    "이름": "최성진",
    "성별": "남",
    "나이": "불명 (20대 후반 추정)",
    "역할": "악당 (연필살인마)",
    "직업": "프리랜서 프로그래머 (추정)",
    "프로필": "온라인 커뮤니티에서 악플을 주도하며 피해자들을 죽음으로 몰아가는 '연필살인마'로, 평범한 외모 뒤에 감춰진 분노와 좌절감이 느껴지며, 익명성에 숨어 타인을 조종하고 파괴하는 데 희열을 느낀다. 과거 악플 피해 경험으로 인해 사회에 대한 깊은 원한을 품고 있으며, 자신의 행위를 '정의 구현'이라고 믿는 왜곡된 신념을 가지고 있다. 온라인 게임 실력이 뛰어나며, 디지털 기기를 능숙하게 다룬다."
  },
  {
    "이름": "김지훈",
    "성별": "남",
    "나이": "20

'[\n  {\n    "이름": "강민준",\n    "성별": "남",\n    "나이": "30대 후반",\n    "역할": "주인공",\n    "직업": "베테랑 형사",\n    "프로필": "과거 강력 사건 트라우마를 극복하며 \'연필살인마\'를 쫓는 냉철하고 직감 있는 형사로, 날카로운 눈빛과 굳게 다문 입술에서 강인함이 느껴지며, 정의감과 책임감이 강하지만 내면에는 깊은 상처를 안고 있다. 동료들과의 관계는 다소 거칠지만, 속으로는 그들을 아끼며, 사건 해결을 위해서는 물불 가리지 않는 열정을 보인다."\n  },\n  {\n    "이름": "박수연",\n    "성별": "여",\n    "나이": "20대 중반",\n    "역할": "조력자",\n    "직업": "해커",\n    "프로필": "뛰어난 해킹 실력으로 강민준을 돕는 조력자로, 과거 악플 피해 경험이 있으며, 앳된 외모와 달리 차분하고 냉철한 성격을 지녔다. 정의감과 연대 의식을 느끼며 사건 해결에 적극적으로 참여하지만, 세상에 대한 불신과 냉소적인 면모도 보인다. 강민준과는 서로의 상처를 보듬어주는 관계로 발전하며, 사건 해결에 중요한 역할을 수행한다."\n  },\n  {\n    "이름": "최성진",\n    "성별": "남",\n    "나이": "불명 (20대 후반 추정)",\n    "역할": "악당 (연필살인마)",\n    "직업": "프리랜서 프로그래머 (추정)",\n    "프로필": "온라인 커뮤니티에서 악플을 주도하며 피해자들을 죽음으로 몰아가는 \'연필살인마\'로, 평범한 외모 뒤에 감춰진 분노와 좌절감이 느껴지며, 익명성에 숨어 타인을 조종하고 파괴하는 데 희열을 느낀다. 과거 악플 피해 경험으로 인해 사회에 대한 깊은 원한을 품고 있으며, 자신의 행위를 \'정의 구현\'이라고 믿는 왜곡된 신념을 가지고 있다. 온라인 게임 실력이 뛰어나며, 디지털 기기를 능숙하게 다룬다."\n  },\n  {\n    "이름": "김지훈",\n    "성별":

## 초안 생성

In [94]:
# 첫 번째 에피소드(초안) 생성
episode1 = novel_gen.create_episode()

에피소드 1화:
 어둠이 짙게 드리운 한강변, 싸늘한 시멘트 바닥에 웅크린 채 강민준은 담배 연기를 내뿜었다. 그의 시선은 희미하게 빛나는 강물 위를 떠돌았다. 며칠째 잠복과 수색, 그리고 끊임없이 되풀이되는 악몽 같은 사건의 잔상들이 그의 뇌리를 괴롭혔다. '연필살인마'. 그 이름만 떠올려도 역겨운 냄새가 코를 찌르는 듯했다.

"강 형사님, 밤샘 근무는 이제 그만하시죠. 이러다 쓰러지십니다."

등 뒤에서 들려오는 걱정스러운 목소리에 강민준은 마지못해 담배를 껐다. 그의 파트너, 김 형사였다. 앳된 얼굴에 어울리지 않게 노련한 수사 감각을 지닌 그는 강민준에게 든든한 버팀목이었다.

"괜찮아. 곧 끝날 거야."

강민준은 짧게 대답하며 몸을 일으켰다. 그의 눈은 여전히 날카롭게 빛나고 있었다. 그는 알고 있었다. 연필살인마는 절대 멈추지 않을 것이라는 것을. 그의 손에 닿지 않는 한, 또 다른 희생자가 나올 것이라는 것을.

사건은 며칠 전, 강남의 한 IT 기업 신입사원이 한강에서 변사체로 발견되면서 시작되었다. 단순 자살로 종결될 뻔했던 사건은 강민준의 예리한 직감에 의해 타살로 전환되었다. 피해자는 생전 온라인 커뮤니티에서 악플에 시달렸으며, 사망 직전까지 극심한 정신적 고통을 겪었다는 사실이 밝혀졌다.

이후 비슷한 수법의 살인사건이 연이어 발생했다. 피해자들은 모두 온라인에서 악플로 고통받던 사람들이었다. 강민준은 이 사건이 단순한 모방범죄가 아닌, '연필살인마'라 불리는 악질적인 연쇄살인범의 소행임을 직감했다.

수사는 난항을 겪었다. 악플러들은 완벽하게 익명 뒤에 숨어 있었고, 증거를 찾기란 하늘의 별따기였다. 하지만 강민준은 포기하지 않았다. 그는 과거의 트라우마를 극복하며 사건에 더욱 몰두했다.

"박수연 씨, 준비는 다 됐나?"

강민준은 무전기를 통해 조력자 박수연에게 연락했다. 그녀는 뛰어난 해킹 실력을 가진 인물로, 강민준의 수사에 결정적인 도움을 주고 있었다.

"네, 형사님. 악플러들의 IP 주소 추적을 완료했습니다. 곧 좌표를 

## 다음화 생성

In [95]:
# 이후 생성할 에피소드는 create_episode()를 호출하면 DB에 저장된 모든 에피소드가 입력값으로 사용됩니다.
episode2 = novel_gen.create_episode()

에피소드 2화:
 강민준은 취조실에 앉아 최성진을 마주했다. 텅 빈 눈동자, 창백한 얼굴. 그는 마치 껍데기만 남은 사람 같았다. "최성진 씨, 당신이 왜 이런 짓을 저질렀는지 말해 주시오." 강민준은 낮고 묵직한 목소리로 물었다. 최성진은 묵묵부답이었다. 그는 마치 아무것도 듣지 못하는 사람처럼 멍하니 허공만 응시했다.

"당신은 악플로 사람들을 죽음으로 몰아갔어. 그들은 당신에게 무슨 잘못을 했나?" 강민준은 다시 한번 물었다. 그의 목소리는 조금 더 강해졌다. 최성진은 여전히 대답하지 않았다. 하지만 그의 눈동자가 미세하게 떨리는 것을 강민준은 놓치지 않았다. "김지훈 씨, 오수진 씨, 그리고 또 다른 피해자들… 그들은 모두 당신의 악플에 시달리다 스스로 목숨을 끊었어. 당신은 그들의 고통을 알면서도 멈추지 않았지."

최성진은 천천히 고개를 들었다. 그의 입가에 희미한 미소가 떠올랐다. "그들은… 벌을 받아야 했어요." 그의 목소리는 쥐어짜는 듯 낮고 쉰 소리였다. "벌? 무슨 벌을 받아야 했다는 거지?" 강민준은 되물었다. 그의 눈은 최성진에게서 한시도 떨어지지 않았다. "그들은… 가짜였어요. 모두 거짓말쟁이였죠." 최성진은 횡설수설하며 중얼거렸다. "가짜? 거짓말쟁이? 그게 무슨 소리야?" 강민준은 그의 말을 이해할 수 없었다.

"그들은… 온라인에서 다른 사람인 척 연기했어요. 착한 척, 정의로운 척… 모두 가짜였죠." 최성진은 점점 더 흥분하기 시작했다. 그의 얼굴은 붉게 상기되었고, 눈은 광기로 번뜩였다. "그래서 당신은 그들을 심판했다는 건가? 익명의 가면을 쓰고 악플을 퍼부어서?" 강민준은 냉소적인 표정으로 물었다. "그렇소. 나는 정의를 구현했을 뿐이오. 그들은… 벌을 받아 마땅했어요." 최성진은 격앙된 목소리로 외쳤다.

강민준은 한숨을 내쉬었다. 그는 최성진의 왜곡된 정의감에 깊은 절망감을 느꼈다. "당신은 정의가 아니야. 당신은 그저 살인마일 뿐이야." 강민준은 단호하게 말했다. 최성진은 그의 말에 더욱 흥분하며 반박했다. "아니오!

In [81]:
episode3 = novel_gen.create_episode()

에피소드 3화:
 결전의 날이 밝았다. 괴식 골목 입구에는 미식 길드 소속 기사들이 굳은 표정으로 도열해 있었다. 갑옷으로 무장한 그들의 모습은 위압적이었고, 손에 든 칼날은 햇빛에 번뜩였다. 그들의 뒤로는 거대한 마차가 줄지어 서 있었는데, 괴식 골목 사람들을 강제로 이주시킬 용도로 준비된 것이었다.

미미는 괴식 골목 사람들과 함께 골목 입구에 섰다. 그녀의 눈빛은 그 어느 때보다 강렬했고, 얼굴에는 비장함이 감돌았다. 리안은 낡은 냄비를 방패 삼아 미미의 옆을 지켰고, 장은 지팡이를 짚으며 굳건한 표정을 지었다. 엘리제는 미식 길드 기사들의 대열을 쏘아보며, 안타까운 표정을 감추지 못했다.

"자, 모두 각오해요! 오늘 우리는 우리의 집을 지키기 위해 싸울 거예요!" 미미의 목소리는 컸고, 그녀의 말은 괴식 골목 사람들의 심장에 용기를 불어넣었다.

미식 길드 기사단의 대장이 앞으로 나섰다. 그는 콧대를 높이 세우고 미미를 내려다보며 차가운 목소리로 말했다. "미미, 마지막 기회를 주겠다. 순순히 물러난다면, 불필요한 유혈 사태는 피할 수 있을 것이다."

미미는 조금도 물러서지 않고 맞받아쳤다. "우리는 아무 데도 가지 않아. 이곳은 우리의 집이고, 우리는 이곳에서 살아갈 권리가 있어."

대장은 코웃음을 쳤다. "어리석은 짓이다. 너희는 미식 길드의 힘을 과소평가하고 있다."

그의 말이 끝나기 무섭게, 미식 길드 기사들은 칼을 뽑아 들고 괴식 골목 사람들에게 달려들었다. 순식간에 골목 입구는 아수라장으로 변했다. 칼날이 부딪히는 소리, 비명 소리, 고함 소리가 뒤섞여 귀를 찢는 듯했다.

리안은 냄비를 휘두르며 용감하게 싸웠지만, 갑옷으로 무장한 기사들을 상대하기에는 역부족이었다. 그는 곧 칼에 맞아 쓰러질 위기에 처했다.

"리안!" 미미는 재빨리 달려가 리안을 감싸 안았다. 그녀의 몸에 칼날이 스치는 순간, 붉은 마력이 뿜어져 나오며 기사들을 튕겨냈다.

미미의 몸에서 뿜어져 나오는 마력은 점점 더 강해졌다. 그녀의 붉은 눈동자는 핏빛으로 물들었

In [82]:
episode4 = novel_gen.create_episode()

에피소드 4화:
 미미의 절규가 멎자, 괴식 골목에는 무거운 침묵이 감돌았다. 핏빛으로 물든 골목길, 쓰러진 사람들의 신음 소리, 그리고 미미의 애끓는 울음만이 귓가를 맴돌았다. 엘리제는 안타까운 눈빛으로 미미를 바라보며, 조용히 그녀의 옆에 섰다. 더 이상 미식 길드 기사들은 공격을 감행하지 못했다. 괴식의 힘에 휩싸인 미미의 모습은 그들에게 공포 그 자체였다.

얼마 후, 미식 길드 기사단의 대장이 앞으로 나섰다. 그는 칼을 거두고, 침통한 표정으로 미미에게 말했다. “미미, 네 뜻은 잘 알았다. 하지만 괴식 골목은 더 이상 존재할 수 없다. 왕도의 안전을 위해서라도, 이곳은 폐쇄되어야 한다.”

미미는 고개를 들지 않았다. 그녀의 시선은 오로지 리안의 싸늘한 시체에만 고정되어 있었다.

“하지만… 너에게 마지막 제안을 하겠다.” 대장은 잠시 망설이다 말을 이었다. “네가 만약 미식 길드에 협력한다면, 괴식 골목 사람들의 이주를 돕고, 그들이 새로운 삶을 시작할 수 있도록 지원하겠다. 또한, 너의 죄는 묻지 않겠다.”

미미는 천천히 고개를 들었다. 그녀의 붉은 눈동자에는 여전히 슬픔과 분노가 가득했지만, 그 속에는 희미한 희망의 빛이 감돌고 있었다.

“미식 길드에 협력하라고…?” 미미의 목소리는 굳게 잠겨 있었다.

“그렇다. 너의 요리 실력과 괴식에 대한 지식은 미식 길드에게 큰 도움이 될 것이다. 너는 미식 길드에서 새로운 요리를 개발하고, 괴식의 긍정적인 가능성을 연구할 수 있다.” 대장은 부드러운 어조로 미미를 설득했다.

미미는 잠시 생각에 잠겼다. 미식 길드에 협력하는 것은 그녀에게 결코 쉬운 일이 아니었다. 미식 길드는 그녀의 가족과 같은 괴식 골목 사람들을 핍박하고, 리안을 죽음으로 몰아넣은 원수였다. 하지만 미미는 괴식 골목 사람들의 미래를 생각하지 않을 수 없었다. 그들은 더 이상 싸울 힘도, 살아갈 희망도 남아 있지 않았다. 미미가 미식 길드에 협력한다면, 그들은 새로운 삶을 시작할 수 있을지도 모른다.

“좋아요.” 미미는 마침내 

In [83]:
episode5 = novel_gen.create_episode()

에피소드 5화:
 폭발과 함께 실험실은 순식간에 화염에 휩싸였다. 찢어지는 듯한 굉음, 자욱한 연기, 그리고 쏟아지는 파편들. 미미는 반사적으로 엘리제를 감싸 안고 몸을 웅크렸다. 폭발의 충격은 엄청났지만, 다행히 마력 방어막 덕분에 직접적인 피해는 피할 수 있었다.

"미미! 괜찮아?!" 엘리제가 외쳤다.

미미는 고개를 끄덕였다. "괜찮아. 하지만… 바르토!"

미미는 바르토를 찾기 위해 주변을 둘러봤지만, 자욱한 연기 때문에 시야가 흐릿했다. 간신히 몸을 일으켜 세운 미미는 주변을 둘러보며 바르토의 이름을 외쳤다.

"바르토 씨! 어디 있어요?!"

"크, 쿨럭… 여기, 여기 있소…"

희미한 신음 소리가 들려왔다. 미미는 소리가 나는 쪽으로 달려갔다. 쏟아지는 파편과 화염을 뚫고 간신히 바르토를 발견했다. 그는 파편에 맞아 쓰러져 있었고, 온몸에는 상처투성이였다.

"바르토 씨! 정신 차려요!" 미미는 바르토를 흔들어 깨웠다.

"미, 미미 씨… 큰일 났소…" 바르토는 힘겹게 말을 이었다. "미식 길드… 미식 길드가 당신을…!"

바르토는 말을 채 잇지 못하고 정신을 잃었다. 미미는 바르토를 안전한 곳으로 옮기고, 그의 상처를 치료하기 시작했다. 엘리제는 주변을 경계하며 혹시 모를 추가 공격에 대비했다.

"대체 무슨 일이 일어난 거지?" 엘리제가 굳은 표정으로 중얼거렸다. "미식 길드가 왜 갑자기 이런 짓을…?"

"함정이야." 미미가 나지막이 말했다. "바르토 씨가 말하려던 건 아마 그거겠지. 미식 길드는 처음부터 나를 제거할 생각이었던 거야."

엘리제는 충격에 휩싸였다. "설마… 그들이 당신을 속였다는 거야? 괴식 골목 사람들을 돕겠다는 약속도 모두 거짓이었어?"

미미는 고개를 끄덕였다. "처음부터 나를 이용하고, 제거할 계획이었던 거야. 괴식 골목 사람들은… 어쩌면 그들도 위험할지 몰라."

미미는 주먹을 꽉 쥐었다. 분노와 슬픔, 그리고 죄책감이 그녀의 마음을 짓눌렀다. 그녀는 미식 길드를 믿은 대가로 너무나 많은 것을 잃었다. 리안의 죽음

In [84]:
episode6 = novel_gen.create_episode()

에피소드 6화:
 미미와 엘리제가 폐허가 된 실험실을 나섰을 때, 왕도는 칠흑 같은 어둠에 잠겨 있었다. 가로등은 꺼져 있었고, 사람들의 모습은 보이지 않았다. 마치 폭풍전야와 같은 고요함이 도시 전체를 감싸고 있었다.

"미식 길드가 도시 전체에 통행 금지령을 내린 것 같아." 엘리제가 주변을 경계하며 말했다. "이제부터는 더욱 조심해야 해. 그들의 눈을 피해 움직이는 건 쉽지 않을 거야."

"괜찮아. 나에게는 숨겨진 통로가 있어." 미미가 자신감 넘치는 목소리로 말했다. "미식 길드의 감시망을 피해, 괴식 골목 사람들에게 연락할 수 있는 유일한 방법이지."

미미는 엘리제를 이끌고 어두운 골목길을 헤쳐 나갔다. 복잡하게 얽힌 골목길은 마치 미로와 같았지만, 미미는 망설임 없이 앞으로 나아갔다. 그녀는 마치 고향에 온 듯 익숙하게 골목길을 누볐다.

"이곳은 예전에 괴식 골목 사람들이 몰래 식재료를 거래하던 곳이야." 미미가 설명했다. "미식 길드의 눈을 피해, 왕도 곳곳으로 연결되는 비밀 통로가 숨겨져 있지."

얼마 후, 미미는 낡은 건물 앞에 멈춰 섰다. 그녀는 건물 벽에 숨겨진 장치를 조작했고, 그러자 건물 벽이 스르륵 열리며 어두운 통로가 나타났다.

"자, 들어가자." 미미가 엘리제를 이끌고 통로 안으로 들어갔다.

통로는 생각보다 깊고 복잡했다. 좁고 어두운 통로를 따라 한참을 걸어가자, 습하고 퀴퀴한 냄새가 코를 찔렀다. 벽에는 곰팡이가 슬어 있었고, 바닥에는 물이 고여 있었다.

"이런 곳을 어떻게 알고 있었어?" 엘리제가 혐오스러운 표정으로 물었다.

"기억은 없지만… 왠지 모르게 익숙해." 미미가 어깨를 으쓱하며 대답했다. "어쩌면 과거에 나도 이곳을 자주 드나들었을지도 모르지."

통로를 따라 계속 나아가던 미미와 엘리제는 마침내 넓은 지하 공간에 도착했다. 그곳에는 낡은 테이블과 의자, 그리고 각종 도구들이 널려 있었다. 한눈에 보기에도 오랫동안 사용되지 않은 듯 먼지가 켜켜이 쌓여 있었다.

"이곳은 괴식 골목 사람들이 비밀리에 모

In [85]:
episode7 = novel_gen.create_episode()

에피소드 7화:
 미미와 엘리제가 향한 곳은 왕궁에서도 가장 깊숙하고 비밀스러운 장소, '미식 창고'였다. 겉으로는 왕족과 귀족들을 위한 고급 식자재를 보관하는 곳이었지만, 실제로는 미식 길드의 비밀 연구소이자, 각종 음모의 근원지였다.

"미식 창고는 엄중하게 경비되고 있어. 일반적인 방법으로는 절대 들어갈 수 없어." 엘리제가 긴장한 표정으로 속삭였다. "하지만 내 신분증과 특별 암호가 있다면, 경비병들의 눈을 속일 수 있어."

엘리제는 품에서 정교하게 만들어진 신분증과 작은 금속 조각을 꺼냈다. 신분증에는 미식 길드 심사관의 표식이 새겨져 있었고, 금속 조각에는 복잡한 문양이 새겨져 있었다.

"이 신분증은 왕족에게 진상되는 식자재를 검수하는 심사관에게만 발급되는 거야." 엘리제가 설명했다. "그리고 이 금속 조각은 미식 창고에 들어갈 수 있는 특별 암호 장치지. 이걸 사용하면, 경비병들의 눈을 속이고 미식 창고 안으로 들어갈 수 있어."

미미는 엘리제가 건네준 신분증과 금속 조각을 주의 깊게 살펴보았다. 그녀는 엘리제의 능숙한 모습에 감탄하면서도, 동시에 불안감을 느꼈다. 미식 길드는 생각보다 훨씬 더 깊숙이 왕궁에 침투해 있었다.

"준비됐어?" 엘리제가 미미를 바라보며 물었다.

미미는 고개를 끄덕였다. "준비됐어. 하지만 무슨 일이 있어도, 내 뒤만 따라와. 무슨 일이 벌어질지 모르니까."

엘리제는 미미를 믿고, 그녀의 뒤를 따랐다. 두 사람은 그림자처럼 조용히 움직이며, 경비병들의 눈을 피해 미식 창고로 향했다.

마침내 미식 창고 앞에 도착한 미미와 엘리제는 숨을 죽였다. 거대한 석조 건물은 웅장하고 위압적인 분위기를 풍겼다. 건물 입구에는 갑옷으로 무장한 경비병들이 날카로운 눈빛으로 주변을 경계하고 있었다.

"자, 이제부터가 진짜야." 엘리제가 긴장한 목소리로 말했다. "내 신분증과 암호 장치를 사용해서 경비병들을 속일 거야. 하지만 조금이라도 수상한 낌새를 보이면, 바로 공격할 준비를 해."

엘리제는 침착하게 신분증을 꺼내 경비병들

In [86]:
episode8 = novel_gen.create_episode()

에피소드 8화:
 휴고가 사용했던 물건들을 꼼꼼히 살펴보았다. 낡은 책상, 먼지 쌓인 서류, 잉크가 말라붙은 펜 등 평범한 물건들뿐이었지만, 미미는 예리한 눈으로 단서를 찾아냈다.

"이걸 봐, 엘리제." 미미가 책상 위에 놓인 세계 지도 한 부분을 가리켰다. "이 지도에 이상한 표시가 되어 있어."

엘리제가 지도를 자세히 살펴보았다. 지도에는 왕도 미식의 위치가 표시되어 있었는데, 그 주변에 붉은색으로 덧칠된 부분이 있었다.

"이건… 대체 뭘 의미하는 거지?" 엘리제가 궁금한 듯 물었다.

"아마 비밀 연구실의 위치를 암시하는 지도일 거야." 미미가 추측했다. "지도의 붉은색 부분을 따라가 보면, 비밀 연구실을 찾을 수 있을지도 몰라."

미미와 엘리제는 지도의 붉은색 부분을 따라 미식 창고 안을 이동하기 시작했다. 그들은 미로처럼 복잡한 창고 통로를 헤쳐 나가며, 붉은색 표시가 가리키는 곳으로 향했다.

마침내 미미와 엘리제는 창고 가장 깊숙한 곳에 위치한 문 앞에 도착했다. 문은 철문으로 굳게 닫혀 있었고, 주변에는 경비병들이 삼엄하게 경비를 서고 있었다.

"여기가 바로 비밀 연구실이야." 엘리제가 긴장한 목소리로 말했다. "하지만 경비가 너무 삼엄해. 어떻게 뚫고 들어가야 하지?"

"나에게 계획이 있어." 미미가 자신감 넘치는 미소를 지으며 말했다. "엘리제, 당신의 신분을 이용할 거야."

미미는 엘리제에게 귓속말로 작전 내용을 설명했다. 엘리제는 잠시 망설이다 고개를 끄덕였다.

엘리제는 당당하게 철문 앞으로 걸어가 경비병들에게 신분증을 보여주었다. 경비병들은 엘리제의 신분을 확인하고, 그녀에게 몇 가지 질문을 던졌다. 엘리제는 능숙하게 대답하며 경비병들을 안심시켰다.

"무슨 일로 오셨습니까?" 경비병 중 한 명이 딱딱한 목소리로 물었다.

"미식 길드에서 특별한 지시를 받고 왔다." 엘리제가 위엄 있는 태도로 대답했다. "지금 당장 비밀 연구실에 들어가야 하니, 문을 열어주십시오."

경비병들은 서로의 얼굴을 쳐다보며 망설였다. 그들은 엘